In [ ]:
import nltk
from nltk import FreqDist
import spacy
import wordcloud
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

import pandas as pd
import re
from unidecode import unidecode

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
"https://gist.github.com/benhoyt/dfafeab26d7c02a52ed17b6229f0cb52"

import collections
import re
import sys
import time


def tokenize(string):
    """Convert string to lowercase and split into words (ignoring
    punctuation), returning list of words.
    """
    return re.findall(r'\w+', string.lower())


def count_ngrams(lines, min_length=2, max_length=4):
    """Iterate through given lines iterator (file object or list of
    lines) and return n-gram frequencies. The return value is a dict
    mapping the length of the n-gram to a collections.Counter
    object of n-gram tuple and number of times that n-gram occurred.
    Returned dict includes n-grams of length min_length to max_length.
    """
    lengths = range(min_length, max_length + 1)
    ngrams = {length: collections.Counter() for length in lengths}
    queue = collections.deque(maxlen=max_length)

    # Helper function to add n-grams at start of current queue to dict
    def add_queue():
        current = tuple(queue)
        for length in lengths:
            if len(current) >= length:
                ngrams[length][current[:length]] += 1

    # Loop through all lines and words and add n-grams to dict
    for line in lines:
        for word in tokenize(line):
            queue.append(word)
            if len(queue) >= max_length:
                add_queue()

    # Make sure we get the n-grams at the tail end of the queue
    while len(queue) > min_length:
        queue.popleft()
        add_queue()

    return ngrams


def print_most_frequent(ngrams, num=50):
    """Print num most common n-grams of each length in n-grams dict."""
    for n in sorted(ngrams):
        print('----- {} most common {}-grams -----'.format(num, n))
        for gram, count in ngrams[n].most_common(num):
            print('{0}: {1}'.format(' '.join(gram), count))
        print('')

In [ ]:
filename = "trustpilot_tripmate.xlsx"
df = pd.read_excel(filename, index_col=0, engine="openpyxl")
df.head()

In [ ]:
# function to remove stopwords
stop_words = [unidecode(w.lower()) for w in stop_words]
def remove_stopwords(rev):
    rev_new = " ".join([i for i in rev.lower().split() if i not in stop_words and len(i)>3])
    return rev_new

df["reviews_cleaned"] = df['text_cleaned_hide_EN'].copy()

# clean reviews
df["reviews_cleaned"] = df['reviews_cleaned'].apply(lambda x: " " + ' '.join([unidecode(w.lower()) for w in x.split()]))

words_toremove = ["lauritz", "lauritzcom", "dba", "tradera", "block", 
                  "auction", "good"]

for word in words_toremove:
    df["reviews_cleaned"] = df["reviews_cleaned"].apply(lambda x: re.sub(f"{word}", " ", x))

# remove stopwords from the text
df["reviews_cleaned"] = df["reviews_cleaned"].apply(remove_stopwords)

df["reviews_cleaned"] = df["reviews_cleaned"].apply(lambda x: re.sub(r"[^a-z\s]", "", x))

df["reviews_cleaned"] = df["reviews_cleaned"].apply(lambda x: re.sub("\s+", " ", x))

In [ ]:
text = " ".join(df[df.rating_star_cleaned_hide < 4]["reviews_cleaned"]).split()

In [ ]:
ngrams = count_ngrams(text)
print_most_frequent(ngrams)